# Module 1
## Data Preprocessing 

### Importing Modules

In [1]:
import pandas as pd
import numpy as np
import json

### Data to be Considered

In [2]:
## Flights Data Pathnames
dirpath = "Data/Flight/"
dirname = "On_Time_On_Time_Performance_"
flightpathname = list()
for i in (2016,2017):
    i = str(i)
    for j in range(1,13):
        j = str(j)
        filename = dirname + i + "_" + j
        pathname = dirpath + i + "/" + filename + "/" + filename + ".csv"
        flightpathname.append(pathname)


## Weather Data
aircodes = ['ATL', 'CLT', 'DEN', 'DFW', 'EWR', 'IAH', 'JFK', 'LAS', 'LAX', 'MCO', 'MIA', 'ORD', 'PHX', 'SEA', 'SFO']

#Weather Factors to consider
weatherfactor = ['date', 'time', 'airport', 'windspeedKmph', 'winddirDegree', 'precipMM', 'visibility', 'pressure', 'cloudcover', 'DewPointF', 'WindGustKmph', 'humidity']

### Processing Flight Data 

In [ ]:
dataframes = list()
for i in flightpathname:   
    dataframe1 = pd.read_csv(i, encoding = 'utf-8', low_memory = False)
    dataframe1  = dataframe1[['FlightDate', 'Quarter' ,'Year' ,'Month' , 'DayofMonth' ,'Origin','CRSDepTime', 'DepTime', 'DepDelay', 'DepDelayMinutes', 'Dest','CRSArrTime','ArrTime', 'ArrDelay', 'ArrDelayMinutes']]
    dataframe1['FlightDate'] = pd.to_datetime(dataframe1['FlightDate'])
    acdatf = list()
    for i in aircodes:
        dtf = dataframe1[(dataframe1.Dest == i)]
        acdatf.append(dtf)
    dataframe1 = pd.concat(acdatf)
    dataframes.append(dataframe1)

In [ ]:
df = pd.concat(dataframes)
df.to_csv("Data/TotalFlightsData.csv", index = False, encoding = 'utf-8')

### Processing Weather Data

In [ ]:
dirpath = "Data/weather/"
weatherdata = dict()
for i in weatherfactor:
    weatherdata[i] = list()
for k in aircodes:
    for i in (2016, 2017):
        i = str(i)
        for j in range(1,13):
            j = str(j)
            p = dirpath + k + "/" + i + "-" + j + ".json"
            data = json.load(open(p))['data']['weather']
            for d in data:
                date = d['date']
                d = d['hourly']
                for t in d:
                    weatherdata['airport'].append(k)
                    weatherdata['date'].append(date)
                    for keys in weatherfactor:
                        try:
                            weatherdata[keys].append(t[keys])
                        except:
                            continue

In [ ]:
wdataf = pd.DataFrame(weatherdata, columns = weatherfactor)
wdataf['date'] = pd.to_datetime(wdataf['date'])
wdataf.to_csv("Data/TotalWeatherData.csv", index = False, encoding = 'utf-8')

### Merging Weather and Flight Data

In [19]:
fdatf = pd.read_csv("Data/TotalFlightsData.csv", encoding = 'utf-8')
wdatf = pd.read_csv("Data/TotalWeatherData.csv", encoding = 'utf-8')
dates = wdatf['date']
dates = dict(dates.value_counts(ascending = True))
dates = [k for k,v in dates.items()]
dates.sort()
time = wdatf['time']
time = dict(time.value_counts())
time = [k for k,v in time.items()]
time.sort()

In [36]:
fdataframe = list()
for i in aircodes:
    fdaircode = fdatf[(fdatf.Dest == i)]
    wdaircode = wdatf[(wdatf.airport == i)]
    airportdatf = list()
    print("Running aircode :", i)
    for j in dates:
        fddates = fdaircode[(fdaircode.FlightDate == j)]
        wddates = wdaircode[(wdaircode.date == j)]
        datedataframes = list()
        for k in range(0, len(time)):
            if time[k] == 2300:
                fdtime = fddates[(fddates.CRSArrTime >= time[k])]
                numrows = len(fdtime.index)
                if numrows == 0:
                    continue
                else:
                    wdtime = wddates[(wddates.time == time[k])]
                    wdtime = wdtime[['windspeedKmph', 'winddirDegree', 'precipMM', 'visibility', 'pressure', 'cloudcover', 'DewPointF', 'WindGustKmph', 'humidity']]
                    wdt = pd.concat([wdtime] * numrows, ignore_index = True)
                    timedatf = pd.concat([fdtime.reset_index(drop = True), wdt.reset_index(drop = True)], axis = 1,ignore_index = True, sort = False)
                    datedataframes.append(timedatf)
            else:
                fdtime = fddates[(fddates.CRSArrTime >= time[k]) & (fddates.CRSArrTime < time[k+1])]
                numrows = len(fdtime.index)
                if numrows == 0:
                    continue
                else:
                    wdtime = wddates[(wddates.time == time[k])]
                    wdtime = wdtime[['windspeedKmph', 'winddirDegree', 'precipMM', 'visibility', 'pressure', 'cloudcover', 'DewPointF', 'WindGustKmph', 'humidity']]
                    wdt = pd.concat([wdtime] * numrows, ignore_index = True)
                    timedatf = pd.concat([fdtime.reset_index(drop = True), wdt.reset_index(drop = True)], axis = 1,ignore_index = True, sort = False)
                    datedataframes.append(timedatf)
        airportdatf.append(pd.concat(datedataframes, ignore_index = True))
    fdataframe.append(pd.concat(airportdatf, ignore_index = True))

Running aircode : ATL
Running aircode : CLT
Running aircode : DEN
Running aircode : DFW
Running aircode : EWR
Running aircode : IAH
Running aircode : JFK
Running aircode : LAS
Running aircode : LAX
Running aircode : MCO
Running aircode : MIA
Running aircode : ORD
Running aircode : PHX
Running aircode : SEA
Running aircode : SFO


In [41]:
finaldatf = pd.concat(fdataframe, ignore_index = True)
finaldatf = finaldatf.rename(columns = {0 : 'FlightDate', 1 : 'Quarter' ,2 : 'Year' ,3 : 'Month' ,4 : 'DayofMonth' ,5 : 'Origin',6 : 'CRSDepTime',7 : 'DepTime',8 : 'DepDelay',9 : 'DepDelayMinutes',10 : 'Dest',11 : 'CRSArrTime',12 : 'ArrTime', 13 : 'ArrDelay',14 : 'ArrDelayMinutes',15 : 'windspeedKmph',16 : 'winddirDegree', 17 : 'precipMM',18 : 'visibility',19 : 'pressure',20 : 'cloudcover', 21 : 'DewPointF', 22 : 'WindGustKmph', 23 : 'humidity'})
finaldatf.to_csv("Data/FinalData.csv", index = False, encoding = "utf-8")

### Generating Training and Test Datasets

In [3]:
dataframe = pd.read_csv("Data/FinalData.csv", encoding = 'utf-8')

In [6]:
trainingdatf = dataframe.iloc[0:5050000]
testdatf = dataframe.iloc[5050000:5055592]
testdatf = dataframe[['FlightDate', 'Quarter' ,'Year' ,'Month' , 'DayofMonth' ,'Origin','CRSDepTime', 'Dest','CRSArrTime','windspeedKmph', 'winddirDegree', 'precipMM', 'visibility', 'pressure', 'cloudcover', 'DewPointF', 'WindGustKmph', 'humidity']]

In [7]:
trainingdatf.to_csv("Data/TrainingData.csv", index = False, encoding = "utf-8")
testdatf.to_csv("Data/TestData.csv", index = False, encoding = "utf-8")